In [309]:
import pandas as pd
import json
import sklearn.svm as svm
from sklearn.linear_model import Lasso, LassoCV
from sklearn.feature_selection import SelectFromModel
import numpy as np
from sklearn.neural_network import MLPClassifier

In [310]:
## Change the name to get your path
user = 'Yash'

In [311]:
def load_Data(user, filename):
   data = json.load(open('dir.json'))
   return (data[user] + filename)

In [312]:
def useable_Data(data, gt, timeStart, timeEnd):
    data['start.time'] = pd.to_datetime(data['start.time'])
    observedData = data[(data['start.time'] >= timeStart) & (data['start.time'] <= timeEnd)]
    observedData['index'] = range(len(observedData))
    gt['index'] = gt['time']
    observedData = pd.merge(observedData,gt[['index','coding']])
    observedData.sort_index(inplace=True)
    copy = observedData[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df']].copy(deep=True)
    copy.loc[-1] = copy.loc[0]  # adding a row
    copy.index = copy.index + 1  # shifting index
    copy.sort_index(inplace=True)
    copy.columns = 'last.' + copy.columns
    observedData = pd.concat([observedData, copy], axis = 1)
    observedData = observedData.drop(observedData.index[len(observedData)-1])
    return observedData

In [313]:
def select_features_from_lasso(X, y, alpha):
    # fit lasso model and pass to select from model
    lasso = LassoCV().fit(X, y)
    model = SelectFromModel(lasso, prefit=True)

    # new features
    X_new = model.transform(X)
    return X.columns[model.get_support()]

In [314]:
def load_struct():
    data = json.load(open('struct.json'))
    return data

In [315]:
def get_observed_data_for_subject(user, subject, files):
    data = pd.read_csv(load_Data(user, files["AG"]), header=0)
    gt1 = pd.read_csv(load_Data(user, files["GT1"]), header=0)
    gt2 = pd.read_csv(load_Data(user, files["GT2"]), header=0)
    type1 = files["GT1"][2]
    type2 = files["GT2"][2]
    start1 = files["StartTime1"]
    end1 = files["EndTime1"]
    start2 = files["StartTime2"]
    end2 = files["EndTime2"]
    observedData = pd.DataFrame(useable_Data(data, gt1, start1, end1))
    observedData['type'] = type1 + '-' + str(subject)
    observedData1 = pd.DataFrame(useable_Data(data, gt2, start2, end2))
    observedData1['type'] = type2 + '-' + str(subject)
    observedData = pd.concat([observedData, observedData1])
    return observedData

In [316]:
def get_all_subjects(user, files):
    observedData = pd.DataFrame()
    for i in files:
        observedData1 = get_observed_data_for_subject(user, 1, struct[str(i)])
        observedData = pd.concat([observedData, observedData1])
        
    return observedData

In [318]:
struct = load_struct()
# observedData1 = get_observed_data_for_subject(user, 1, struct["1"])
observedData = get_all_subjects(user, struct)

/Users/yashmehra/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


FileNotFoundError: File b'/Users/yashmehra/Google Drive/Data Capstone /Project Folder/Cleaned_Data/GTA13-02.csv' does not exist

In [302]:
observedData.shape

(28807, 19)

In [288]:
observedData.columns

Index(['Unnamed: 0', 'coding', 'dfreq', 'index', 'last.dfreq', 'last.mean.ang',
       'last.mean.vm', 'last.p625', 'last.ratio.df', 'last.sd.ang',
       'last.sd.vm', 'mean.ang', 'mean.vm', 'p625', 'ratio.df', 'sd.ang',
       'sd.vm', 'start.time', 'type'],
      dtype='object')

## Build test and train Datasets

In [303]:
observedData = observedData.sample(frac = 1)

train = observedData[:21600]
test = observedData[21600:]

X = observedData[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df', 'last.mean.vm', 'last.sd.vm', 'last.mean.ang', 'last.sd.ang', 'last.p625', 'last.dfreq', 'last.ratio.df']]
y = observedData['coding']

trainX = train[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df', 'last.mean.vm', 'last.sd.vm', 'last.mean.ang', 'last.sd.ang', 'last.p625', 'last.dfreq', 'last.ratio.df']]
trainY = train['coding']
testX = test[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df', 'last.mean.vm', 'last.sd.vm', 'last.mean.ang', 'last.sd.ang', 'last.p625', 'last.dfreq', 'last.ratio.df']]
testY = test['coding']

## Lasso (Feature selection)

In [168]:
# trainY = (trainY == 'sedentary').astype(int)
# testY = (testY == 'sedentary').astype(int)

In [304]:
# features = select_features_from_lasso(trainX, trainY, 0.001)

In [188]:
trainX = trainX[features]
testX = testX[features]

## SVM

In [305]:
clf = svm.SVC(cache_size=7000)
clf.fit(trainX, trainY)

SVC(C=1.0, cache_size=7000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [306]:
clf.score(testX,testY)

0.7878451505480782

## Neural Network

In [307]:
n_net = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2000, 1000), random_state=1, verbose = True)
n_net.fit(trainX,trainY)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(2000, 1000), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [308]:
np.sum(n_net.predict(testX) == testY) / len(testX)

0.8065769390869988